## Text Summarization Basic -> SOA Models

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [18]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=736752b52e129897db12240e1256e10075470e1eec30498e33e68dd331529762
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
from transformers import pipeline, set_seed

## Load Data

In [6]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Features: ['article', 'highlights', 'id']


In [7]:
sample = dataset["train"][34]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 3262):

PHUKET, Thailand (CNN) -- Relatives of the 89 people killed in Sunday's plane crash in Phuket continued to arrive in the Thai resort town on Tuesday to try to identify their loved ones' remains. A Buddhist monk blesses the wreckage of One-Two-Go airline's MD-82 jet at Phuket airport on Tuesday. Most of those killed were foreign nationals, and about 36 of the bodies still have yet to be identified, according to Thailand's state-run Thai News Agency (TNA). All of the unidentified victims were fore

Summary (length: 311):
Relatives of the 89 people killed in Thai air crash are still arriving in Phuket .
About 36 of the bodies have yet to be identified, according to Thai authorities .
U.S. Embassy in Bangkok confirms that five Americans were among the dead .
The investigation into what caused Sunday's crash is still continuing .


In [8]:
# We'll collect the generated summaries of each model in a dictionary
summaries = {}

## Model Building

### NLTK

In [9]:
sample_text = dataset["train"][34]["article"][:2000]

In [10]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [12]:
summaries["baseline"] = three_sentence_summary(sample_text)

### BART

In [13]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### T5 Large

In [21]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [14]:
print("GROUND TRUTH")
print(dataset["train"][34]["highlights"])
print("")

GROUND TRUTH
Relatives of the 89 people killed in Thai air crash are still arriving in Phuket .
About 36 of the bodies have yet to be identified, according to Thai authorities .
U.S. Embassy in Bangkok confirms that five Americans were among the dead .
The investigation into what caused Sunday's crash is still continuing .



In [22]:
for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

BASELINE
PHUKET, Thailand (CNN) -- Relatives of the 89 people killed in Sunday's plane crash in Phuket continued to arrive in the Thai resort town on Tuesday to try to identify their loved ones' remains.
A Buddhist monk blesses the wreckage of One-Two-Go airline's MD-82 jet at Phuket airport on Tuesday.
Most of those killed were foreign nationals, and about 36 of the bodies still have yet to be identified, according to Thailand's state-run Thai News Agency (TNA).

BART
One-Two-Go jet crashed in bad weather as it landed at Phuket's airport on Sunday.
Most of those killed were foreign nationals, and about 36 of the bodies still have yet to be identified.
Investigation into what caused the crash continues, but strong winds and rain are a suspected factor.

T5
relatives of 89 people killed in plane crash in Phuket continue to arrive in town .
about 36 of the bodies still have yet to be identified, according to state media .
u.s. embassy in Bangkok said that five americans were among those 

## Model Evaluation

In [19]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [23]:
reference = dataset["train"][34]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.470588,0.268657,0.338235,0.455882
bart,0.476190,0.213592,0.323810,0.457143
t5,0.611111,0.433962,0.574074,0.611111
